## Bases de datos con sql

In [44]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from mpl_toolkits.mplot3d import Axes3D
import requests
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from pandasql import sqldf
from pandasql import load_births

### Podemos hacer uso de bases de datos por medio de pandasql

In [2]:
bir = load_births()
print(sqldf("SELECT * FROM bir where births > 250000 limit 5;", locals()))

                         date  births
0  1975-01-01 00:00:00.000000  265775
1  1975-03-01 00:00:00.000000  268849
2  1975-05-01 00:00:00.000000  254545
3  1975-06-01 00:00:00.000000  254096
4  1975-07-01 00:00:00.000000  275163


In [3]:
q = """select date(date) as Fecha, sum(births) as "Nacimientos Totales" from bir group by date limit 10; """
sqldf(q,locals())

,Fecha,Nacimientos Totales
0,1975-01-01,265775
1,1975-02-01,241045
2,1975-03-01,268849
3,1975-04-01,247455
4,1975-05-01,254545
5,1975-06-01,254096
6,1975-07-01,275163
7,1975-08-01,281300
8,1975-09-01,270738
9,1975-10-01,265494


### Podemos hacer uso de una función para imprimir tablas

In [12]:
def sqldff(q):
    return sqldf(q,globals())

In [5]:
q = """select date(date) as Fecha, sum(births) as "Nacimientos Totales" from bir group by date limit 5; """
sqldff(q)

,Fecha,Nacimientos Totales
0,1975-01-01,265775
1,1975-02-01,241045
2,1975-03-01,268849
3,1975-04-01,247455
4,1975-05-01,254545


In [26]:
tab=[]
df = []
fix=["26","2226","2227","30","69"]
fix2 = ["s","n","c"]
u = "https://www.indexmundi.com/map/?t=0&v="
u2 = "&r="
u3 = "a&l=en"
for i in range(5):
    for j in range(3):
        aux = u,fix[i],u2,fix2[j],u3
        url = "".join(aux)
        html = requests.get(url).content                                  
        df_list = pd.read_html(html)                                      
        df.append(df_list[1])
    tab.append(pd.concat([df[0],df[1],df[2]], ignore_index=True))
    df=[]
    tab[i] = tab[i].sort_values("Country", ascending = True)

In [69]:
tabla = tab[0]
for i in range(1,5):
    tabla = pd.merge(tabla,tab[i], on = "Country",how="inner")
tabla = pd.DataFrame({'Pais':tabla["Country"], 'Tasa_de_muerte':tabla["Death rate (deaths/1,000 population)"],
                     'Medicos':tabla["Physicians density (physicians/1,000 population)"],
                     'Camas_de_Hospital':tabla["Hospital bed density (beds/1,000 population)"],
                     'Esperanza_de_Vida':tabla["Life expectancy at birth (years)"],
                     'Pobreza':tabla["Population below poverty line (%)"]})
aux = tabla.sort_values("Esperanza_de_Vida", ascending = False)

In [60]:
aux.set_index('Pais',inplace=True)
aux.dtypes

Tasa_de_muerte       int64
Medicos              int64
Camas_de_Hospital    int64
Esperanza_de_Vida    int64
Pobreza              int64
dtype: object

### Podemos hacer uso de nuestros propios datos para manipularlos con sql

In [72]:
q = """select Pais, Tasa_de_muerte, Esperanza_de_Vida from tabla order by Pais limit 5"""
sqldff(q)

,Pais,Tasa_de_muerte,Esperanza_de_Vida
0,Argentina,8,78
1,"Bahamas, The",7,73
2,Belize,4,75
3,Bolivia,6,70
4,Brazil,7,74


### Usando el comando Join, tenemos distintas opciones para unir tablas

In [55]:
c = tab[0]
s = tab[1]
q = """select * from c inner join s on (c.Country = s.Country) limit 8"""
sqldff(q)

,Country,"Death rate (deaths/1,000 population)",Year,Country,"Physicians density (physicians/1,000 population)",Year
0,Antigua and Barbuda,6,2018,Antigua and Barbuda,3,2017
1,Argentina,8,2018,Argentina,4,2017
2,"Bahamas, The",7,2018,"Bahamas, The",2,2017
3,Barbados,9,2018,Barbados,2,2017
4,Belize,4,2018,Belize,1,2017
5,Bolivia,6,2018,Bolivia,2,2016
6,Brazil,7,2018,Brazil,2,2018
7,Canada,9,2018,Canada,3,2017


## Podemos usar sqlite para crear bases de datos

In [37]:
db = sqlite3.connect(':memory:') #Conectando a las bases de datos de sqlite
cur = db.cursor()

In [38]:
cur.execute('''CREATE TABLE books(ID INTEGER PRIMARY KEY, Pais TEXT, "Tasa de muerte" INTEGER, "Pobreza" INTEGER, Año TEXT)''')
db.commit()

In [39]:
cur.execute('''INSERT INTO books values(1, "Canada", "9", 4, 2015)''')
cur.execute('''INSERT INTO books values(2, "Mexico", "8", 5, 2016)''')
cur.execute('''INSERT INTO books values(3, "Argentina", "8", 5, 2016)''')
db.commit()

In [40]:
dbooks = cur.execute('''select * from books;''').fetchall()
db.commit()
print(dbooks)

[(1, 'Canada', 9, 4, '2015'), (2, 'Mexico', 8, 5, '2016'), (3, 'Argentina', 8, 5, '2016')]


### Y con ayuda de Panda podemos darles formato de tabla.

In [41]:
dbooks = pd.read_sql_query("SELECT * from books;", db)
dbooks

,ID,Pais,Tasa de muerte,Pobreza,Año
0,1,Canada,9,4,2015
1,2,Mexico,8,5,2016
2,3,Argentina,8,5,2016


In [43]:
cur.execute('''DROP TABLE books''') #El comando DROP, borrará la tabla
db.commit()

OperationalError: no such table: books